In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
all_covs = db.collect_all_coverages(session)

In [3]:
annual_winter_tdd_covs = db.collect_all_coverages(
    session, 
    configuration_parameter_values_filter=[
        db.get_configuration_parameter_value_by_names(session, "historical_variable", "tdd"),
        db.get_configuration_parameter_value_by_names(session, "aggregation_period", "annual"),
        db.get_configuration_parameter_value_by_names(session, "observation_year_period", "winter")
    ]
)

In [4]:
[cov.identifier for cov in annual_winter_tdd_covs]

['tdd_annual_seasonal-annual-historical-tdd-absolute-winter']

In [9]:
cov = db.get_coverage(session, "tdd_annual_seasonal-annual-historical-tdd-absolute-winter")

In [10]:
cov.configuration.coverage_id_pattern

'{name}-{aggregation_period}-{archive}-{historical_variable}-{measure}-{observation_year_period}'

In [11]:
cov.configuration.get_thredds_url_fragment(cov.identifier)

'cline_yr/TDd_S01_*.nc'